# Amsi-Bypass-Powershell
git@github.com:S3cur3Th1sSh1t/Amsi-Bypass-
Powershell.git

## extract ps1 from readme.md

In [2]:
import os
import re

# ====== Config ======
input_file = './Amsi-Bypass-Powershell/README.md'            
output_dir = './Amsi-Bypass-Powershell/generated_ps1'         
os.makedirs(output_dir, exist_ok=True)

# ====== Read README content ======
with open(input_file, 'r', encoding='utf-8') as f:
    content = f.read()

# ====== Find all powershell code blocks ======
pattern = re.compile(r'```powershell\s*\n(.*?)```', re.DOTALL | re.IGNORECASE)
matches = pattern.findall(content)

print(f"✅ Tổng số đoạn script tìm được: {len(matches)}")

# ====== Process and save each script ======
valid_script_count = 0

for idx, script in enumerate(matches, 1):
    script = script.strip()

    # Bỏ qua những đoạn quá ngắn (không đáng để lưu)
    if len(script) < 10:
        continue

    # Nếu trong script có comment ghi filename, lấy luôn
    filename_match = re.search(r'#\s*Filename\s*:\s*(.+?\.ps1)', script, re.IGNORECASE)
    if filename_match:
        filename = filename_match.group(1).strip()
    else:
        filename = f'bypass_{idx:03d}.ps1'

    output_path = os.path.join(output_dir, filename)

    # Ghi file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(script)

    valid_script_count += 1
    print(f"✅ Saved {output_path}")

print(f"\n🎯 Hoàn thành: Đã tách được {valid_script_count} script hợp lệ!")

✅ Tổng số đoạn script tìm được: 10
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_001.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_002.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_003.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_004.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_005.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_006.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_007.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_008.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_009.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_010.ps1

🎯 Hoàn thành: Đã tách được 10 script hợp lệ!


## convert ps1 -> csv

In [4]:
import os
import csv

# ====== Config ======
input_folder = './Amsi-Bypass-Powershell/generated_ps1'  # Folder chứa các .ps1
output_csv = './dataset/Amsi-Bypass-Powershell.csv'

# ====== Build payloads ======
data_list = []

for filename in os.listdir(input_folder):
    if filename.endswith('.ps1'):
        file_path = os.path.join(input_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            b64_decoded = f.read().strip()

        # === Áp dụng giống logic combine_attributes ===
        path_exec = "powershell"
        arguments = ""  # Không có arguments
        full_payload = f"{path_exec} {arguments} {b64_decoded}".strip()

        data = {
            'Payload': full_payload,
            'Label': 'Bypass'
        }
        data_list.append(data)

# ====== Save CSV ======
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Payload', 'Label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

print(f"✅ Đã lưu {len(data_list)} samples vào {output_csv}")

✅ Đã lưu 10 samples vào ./dataset/Amsi-Bypass-Powershell.csv


# mpsd

In [ ]:
import os 
import hashlib
import time
import pandas as pd
import requests
from dotenv import load_dotenv

# ===== Cấu hình =====
input_dir = './mpsd/malicious_pure'
output_csv = './dataset/mpsd.csv'
unknown_csv = './checkpoint/mpsd/mpsd_label_unknown.csv'
checkpoint_file = './checkpoint/mpsd/mpsd_checkpoint.txt'

load_dotenv()
api_keys = [os.getenv(f'api{i}') for i in range(1, 10) if os.getenv(f'api{i}')]

# ===== Hàm hỗ trợ =====
def get_sha256(content: bytes) -> str:
    return hashlib.sha256(content).hexdigest()

def map_label_from_vt(vt_name: str, categories: list) -> str:
    name = (vt_name or "").lower()
    cats = [c.lower() for c in categories] if categories else []

    BYPASS_KEYWORDS = ['bypass', 'disable defender', 'evade', 'avoid', 'kill defender', 'anti-av']
    TASKEXEC_KEYWORDS = ['schtask', 'task', 'scheduled', 'com', 'wmi', 'bitsadmin', 'bits', 'vb task', 'dynamite']
    DOWNLOADER_KEYWORDS = ['downloader', 'dropper', 'transfer', 'webdl', 'fetch', 'grabber', 'iexds', 'dfsp', 'curl', 'wget', 'powershellhttp']
    INJECTOR_KEYWORDS = ['inject', 'shellcode', 'reflective', 'pe loader', 'runpe', 'rhttp', 'remote dll', 'trojan', 'hollows', 'process hollowing']
    PAYLOAD_KEYWORDS = ['empire', 'veil', 'power', 'txt c2', 'powershell backdoor', 'remcos', 'reverse shell']

    def match(keywords):
        return any(kw in name for kw in keywords) or any(kw in c for kw in keywords for c in cats)

    if match(BYPASS_KEYWORDS):
        return 'Bypass'
    elif match(TASKEXEC_KEYWORDS):
        return 'TaskExecution'
    elif match(DOWNLOADER_KEYWORDS):
        return 'Downloader'
    elif match(INJECTOR_KEYWORDS):
        return 'Injector'
    elif match(PAYLOAD_KEYWORDS):
        return 'Payload'
    return 'Unknown'

# ===== Checkpoint =====
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        labeled_set = set(line.strip() for line in f)
else:
    labeled_set = set()

# ===== Khởi tạo =====
ps1_files = [f for f in os.listdir(input_dir) if f.endswith('.ps1')]
results = []
unknowns = []
api_index = 0

# ===== Xử lý từng file =====
for fname in ps1_files:
    fpath = os.path.join(input_dir, fname)
    with open(fpath, 'rb') as f:
        content_bytes = f.read()

    sha256 = get_sha256(content_bytes)
    if sha256 in labeled_set:
        continue

    content_str = content_bytes.decode('utf-8', errors='ignore')
    current_key = api_keys[api_index]
    api_index = (api_index + 1) % len(api_keys)

    url = f"https://www.virustotal.com/api/v3/files/{sha256}"
    headers = {"x-apikey": current_key}

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 404:
            label = 'Unknown'
        elif r.status_code == 403:
            print(f"❌ API key bị giới hạn: {current_key}")
            break
        elif r.status_code == 429:
            print(f"🚫 Quá giới hạn API (429) cho: {sha256}")
            continue
        elif r.status_code != 200:
            print(f"⚠️ Lỗi khi gọi {sha256}: {r.status_code}")
            continue
        else:
            data = r.json()
            name = data.get('data', {}).get('attributes', {}).get('meaningful_name', '')
            tags = data.get('data', {}).get('attributes', {}).get('tags', [])
            label = map_label_from_vt(name, tags)

        # ✅ Ghi checkpoint nếu không bị lỗi 403/429
        with open(checkpoint_file, 'a') as f:
            f.write(sha256 + '\n')

    except Exception as e:
        print(f"⚠️ Lỗi khi gọi VT: {e}")
        continue

    row = {
        'SHA256': sha256,
        'Full Payload': content_str,
        'Label': label
    }

    row_df = pd.DataFrame([row])

    if label == 'Unknown':
        row_df.to_csv(unknown_csv, mode='a', header=not os.path.exists(unknown_csv), index=False, escapechar='\\')
    else:
        row_df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False, escapechar='\\')

    print(f"✅ {fname} → {label}")
    time.sleep(2)

# ===== Lưu kết quả =====
if results:
    df = pd.DataFrame(results)
    if os.path.exists(output_csv):
        df.to_csv(output_csv, mode='a', header=False, index=False, escapechar='\\')
    else:
        df.to_csv(output_csv, index=False, escapechar='\\')
    print(f"✅ Đã lưu {len(results)} nhãn hợp lệ → {output_csv}")

if unknowns:
    df = pd.DataFrame(unknowns)
    if os.path.exists(unknown_csv):
        df.to_csv(unknown_csv, mode='a', header=False, index=False, escapechar='\\')
    else:
        df.to_csv(unknown_csv, index=False, escapechar='\\')
    print(f"ℹ️ Đã lưu {len(unknowns)} mẫu Unknown → {unknown_csv}")

if not results and not unknowns:
    print("👍 Không có file mới cần xử lý.")


✅ 3418.ps1 → Payload
✅ 3021.ps1 → Unknown
✅ 1479.ps1 → Payload
✅ 2650.ps1 → Unknown
✅ 193.ps1 → Payload
✅ 2108.ps1 → Unknown
✅ 481.ps1 → Unknown
✅ 3647.ps1 → Unknown
✅ 1037.ps1 → Payload
✅ 3740.ps1 → Payload
✅ 942.ps1 → Unknown
✅ 3772.ps1 → Unknown
✅ 2885.ps1 → Payload
✅ 909.ps1 → Unknown
✅ 1660.ps1 → Unknown
✅ 2174.ps1 → Unknown
✅ 2770.ps1 → Unknown
✅ 1288.ps1 → Payload
✅ 2243.ps1 → Unknown
✅ 2519.ps1 → Payload
✅ 2309.ps1 → Payload
✅ 438.ps1 → Payload
✅ 1988.ps1 → Unknown
✅ 844.ps1 → Unknown
✅ 2139.ps1 → TaskExecution
✅ 575.ps1 → Unknown
✅ 2811.ps1 → Unknown
✅ 2343.ps1 → Unknown
✅ 3597.ps1 → Unknown
✅ 3721.ps1 → Unknown
✅ 1302.ps1 → Unknown
✅ 82.ps1 → Payload
✅ 1992.ps1 → Unknown
✅ 2743.ps1 → Unknown
✅ 1672.ps1 → Unknown
✅ 395.ps1 → Unknown
✅ git_18.ps1 → Payload
✅ 404.ps1 → Payload
✅ 3067.ps1 → Unknown
✅ 3457.ps1 → Payload
✅ 3949.ps1 → Unknown
✅ 2906.ps1 → Payload
✅ 3893.ps1 → Unknown
✅ 2088.ps1 → Payload
✅ 3099.ps1 → Unknown
✅ 1920.ps1 → Unknown
✅ 2239.ps1 → Unknown
✅ 4.ps1 → Payloa

KeyboardInterrupt: 

## run unknown again

In [1]:
import os
import pandas as pd
import requests
import time
from dotenv import load_dotenv

unknown_csv = './mpsd/label_unknown.csv'
output_csv = './dataset/mpsd.csv'
checkpoint_file = './mpsd/labeled_sha256.txt'

load_dotenv()
api_keys = [os.getenv(f'api{i}') for i in range(1, 10) if os.getenv(f'api{i}')]

def map_label_from_vt(vt_name: str, categories: list) -> str:
    name = (vt_name or "").lower()
    cats = [c.lower() for c in categories] if categories else []

    BYPASS_KEYWORDS = ['bypass', 'disable defender', 'evade', 'avoid', 'kill defender', 'anti-av']
    TASKEXEC_KEYWORDS = ['schtask', 'task', 'scheduled', 'com', 'wmi', 'bitsadmin', 'bits', 'vb task', 'dynamite']
    DOWNLOADER_KEYWORDS = ['downloader', 'dropper', 'transfer', 'webdl', 'fetch', 'grabber', 'iexds', 'dfsp', 'curl', 'wget', 'powershellhttp']
    INJECTOR_KEYWORDS = ['inject', 'shellcode', 'reflective', 'pe loader', 'runpe', 'rhttp', 'remote dll', 'trojan', 'hollows', 'process hollowing']
    PAYLOAD_KEYWORDS = ['empire', 'veil', 'power', 'txt c2', 'powershell backdoor', 'remcos', 'reverse shell']

    def match(keywords):
        return any(kw in name for kw in keywords) or any(kw in c for kw in keywords for c in cats)

    if match(BYPASS_KEYWORDS):
        return 'Bypass'
    elif match(TASKEXEC_KEYWORDS):
        return 'TaskExecution'
    elif match(DOWNLOADER_KEYWORDS):
        return 'Downloader'
    elif match(INJECTOR_KEYWORDS):
        return 'Injector'
    elif match(PAYLOAD_KEYWORDS):
        return 'Payload'
    return 'Unknown'

# Đọc dữ liệu Unknown
df = pd.read_csv(unknown_csv)

# Chuẩn bị
api_index = 0
updated_rows = []
still_unknown = []

# Xử lý lại từng dòng
for _, row in df.iterrows():
    sha256 = row['SHA256']
    payload = row['Full Payload']

    current_key = api_keys[api_index]
    api_index = (api_index + 1) % len(api_keys)
    url = f"https://www.virustotal.com/api/v3/files/{sha256}"
    headers = {"x-apikey": current_key}

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 403:
            print(f"❌ API key bị block: {current_key}")
            break
        elif r.status_code == 429:
            print(f"🚫 Quá giới hạn API: {sha256}")
            continue
        elif r.status_code == 404:
            label = 'Unknown'
        elif r.status_code != 200:
            print(f"⚠️ Lỗi không xác định {sha256}: {r.status_code}")
            continue
        else:
            data = r.json()
            name = data.get('data', {}).get('attributes', {}).get('meaningful_name', '')
            tags = data.get('data', {}).get('attributes', {}).get('tags', [])
            label = map_label_from_vt(name, tags)

        row['Label'] = label
        if label == 'Unknown':
            still_unknown.append(row)
        else:
            updated_rows.append(row)
            with open(checkpoint_file, 'a') as f:
                f.write(sha256 + '\n')

        print(f"🔁 {sha256} → {label}")
        time.sleep(2)

    except Exception as e:
        print(f"⚠️ Lỗi khi gọi VT cho {sha256}: {e}")
        continue

# Lưu kết quả mới
if updated_rows:
    pd.DataFrame(updated_rows).to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False, escapechar='\\')
    print(f"✅ Đã ghi {len(updated_rows)} mẫu có nhãn mới vào {output_csv}")

if still_unknown:
    pd.DataFrame(still_unknown).to_csv(unknown_csv, index=False, escapechar='\\')
    print(f"⚠️ Vẫn còn {len(still_unknown)} mẫu Unknown → lưu lại {unknown_csv}")
else:
    print("🎉 Tất cả mẫu Unknown đã được xử lý!")



🔁 89b086ed34eda89863ceff55a274adf06c73621e05cb1b71ed72b41426e0c274 → Unknown
🔁 c9b2459fa8de4248a4f031b52c4fc5b47ba156b147a9431fb39fe809488b4936 → Unknown
🔁 a39acda83a9cb4f9c214240ebd45180219dc2d18c43793983f5185e9d4557d98 → Unknown
🔁 36891308b53d9fd404b966ba48d7ec065602225f9b786ef4046be0f2b5624d04 → Unknown
🔁 f41239219b788cf2b79ff637a6f09522f574f194d62a40cc784db65cdbd3f3a8 → Unknown
🔁 3b78b1dde2eb7c6db830413277f3c8fc7f42bad00ebc1d7d2978dc0dba0ab938 → Unknown
🔁 8877527aeae3de3f75e94b6930aa162a315cc0c93bba175c16460e80b7531488 → Unknown
🔁 d5389cd50f8688dadea325e548e9dc5832a608dba4273c2ed80f0cad59463114 → Unknown
🔁 ae4a8c87bc4061b5e225bd7ec5da313627faef06fcf4a403943e07dfb3fb3deb → Unknown
🔁 47e361b57dacc6be238b2b8af06efdb45d3a5e665ef7b15ad3f36585d5eedcb7 → Unknown
🔁 de0050d9c610391ff29643407eb4aaa40254c16cf99d526f641738d8e46fa77d → Unknown
🔁 3f64ba46a3bcb06bb24e6cde6d24609821973810c93ea0fa224480f8ecffb2ab → Unknown
🔁 d973ac46cec58b4fe9d50302fbf6e78761591ad5dc80b1b8a41bd6d12b82bb6b → Unknown